In [1]:
import re
import os
import zipfile
import pysolr

from datetime import datetime
from pprint import pprint

In [2]:
# Solr collection url 
SOLR_URL = 'http://solr:8983/solr/cmp269'

# Where to find the data files
BASE_PATH = 'data/FSP95'

In [3]:
# unzip files to data/FSP95
zipf = zipfile.ZipFile("data/FSP95.zip", "r")
zipf.extractall("data")
zipf.close()

In [4]:
def parse_doc(doc_as_string):
    """Converts FSP string doc into dict"""

    doc_tpl = {
        'DOCNO': 'docno_s',
        'DOCID': 'docid_s',
        'DATE': 'date_dt',
        'CATEGORY': 'category_s',
        'TEXT': 'text_txt_pt',
    }

    doc = {}

    for key in doc_tpl.keys():
        # generates something like:
        # str_pattern = ".*\<DOCID\>(.*)\<\/DOCID\>.*"
        str_pattern = ".*\<%s\>(.*)\<\/%s\>.*" % (key, key)

        # compiles pattern, capture match, updates doc
        re_pattern = re.compile(str_pattern, re.DOTALL)
        match = re_pattern.match(doc_as_string)
        
        if match:
            value = match.group(1)
            value = value.replace("\n", "")

            if key == 'DATE':
                date_obj = datetime.strptime(value, '%y%m%d')
                solr_date_str = date_obj.strftime("%Y-%m-%dT%H:%m:%S")
                value = solr_date_str

            solr_key = doc_tpl[key]
            doc.update({solr_key: value})

    # ensures we have all keys..
    for key in doc_tpl.values():
        assert(doc[key] is not None)

    return doc

In [5]:
def parse_file(fpath, doc_tag):
    '''Reads file line by line and extract the docs'''

    docs = []
    closing_doc_line = "</%s>\n" % doc_tag
    
    with open(fpath, encoding="iso-8859-1") as file:
        doc_as_string = ""

        line = file.readline()
        while (line):
            if (line) == closing_doc_line:
                doc_as_string += line
                
                doc = parse_doc(doc_as_string)
                docs.append(doc)

                doc_as_string = ""
            else:
                doc_as_string += line

            line = file.readline()

    return docs

In [6]:
# testing...
docs = parse_file(os.path.join(BASE_PATH, 'FSP.950101.sgml'), 'DOC')
pprint(docs[0])

{'category_s': 'PRIMEIRA_PÁGINA',
 'date_dt': '1995-01-01T00:01:00',
 'docid_s': 'FSP950101-001',
 'docno_s': 'FSP950101-001',
 'text_txt_pt': 'Datafolha revela que 70% esperam governo ótimo ou bom; '
                'Collor teve 71% antes de assumir a Presidência O sociólogo '
                'Fernando Henrique Cardoso, 63, toma possa hoje às 16h30 no '
                'Congresso como o 38º presidente do Brasil. Eleito em 3 de '
                'outubro com 34,3 milhões de votos, 54,28% dos válidos, FHC '
                'sucede a Itamar Augusto Cautiero Franco, 63, de quem foi '
                'ministro das Relações Exteriores e da Fazenda.O Datafolha '
                'revela que 79% apóiam o real, moeda lançada pela equipe de '
                'FHC. Para 70%, seu governo será ótimo ou bom. Em 90, 71% '
                'tinham a mesma expectativa em relação a Fernando Collor.O '
                'programa de FHC definiu como prioridades a manutenção do '
                'real e as r

In [7]:
# Defines some solr helpers..
si = pysolr.Solr(SOLR_URL)

def index_docs(docs):
    if not isinstance(docs, list):
        docs = [docs, ]
    
    ret = si.add(docs)

    return ret

def remove_docs(query):
    ret = si.delete(q=query)
    return ret


In [8]:
# Empty collection
remove_docs("*:*")

# Index all docs
files = os.listdir(BASE_PATH)

for fpath in files:

    if not fpath.endswith("sgml"):
        continue
    
    full_fpath = os.path.join(BASE_PATH, fpath)
    docs = parse_file(full_fpath, 'DOC')

    index_docs(docs)
    print("Indexados %d docs do arquivo %s" % (len(docs), fpath))

print("The end")

Indexados 167 docs do arquivo FSP.950101.sgml
Indexados 125 docs do arquivo FSP.950102.sgml
Indexados 112 docs do arquivo FSP.950103.sgml
Indexados 114 docs do arquivo FSP.950104.sgml
Indexados 124 docs do arquivo FSP.950105.sgml
Indexados 137 docs do arquivo FSP.950106.sgml
Indexados 93 docs do arquivo FSP.950107.sgml
Indexados 166 docs do arquivo FSP.950108.sgml
Indexados 88 docs do arquivo FSP.950109.sgml
Indexados 127 docs do arquivo FSP.950110.sgml
Indexados 130 docs do arquivo FSP.950111.sgml
Indexados 141 docs do arquivo FSP.950112.sgml
Indexados 119 docs do arquivo FSP.950113.sgml
Indexados 101 docs do arquivo FSP.950114.sgml
Indexados 164 docs do arquivo FSP.950115.sgml
Indexados 99 docs do arquivo FSP.950116.sgml
Indexados 119 docs do arquivo FSP.950117.sgml
Indexados 125 docs do arquivo FSP.950118.sgml
Indexados 136 docs do arquivo FSP.950119.sgml
Indexados 128 docs do arquivo FSP.950120.sgml
Indexados 116 docs do arquivo FSP.950121.sgml
Indexados 185 docs do arquivo FSP.950